# Heat flux with trasient albedo
- This script is used to export heat flux variables;
- Simulations: CNTL, ROOF_TV, IMPRAOD_TV, WALL_TV;

In [149]:
import os
import xarray as xr
import numpy as np
import pandas as pd
import cftime

In [182]:
path = '/work/n02/n02/yuansun/cesm/'
# surface
sfile = path + 'cesm_inputdata/lnd/clm2/surfdata_map/release-clm5.0.18/surfdata_0.9x1.25_hist_16pfts_Irrig_CMIP6_simyr1850_c190214.nc'
sf = xr.open_dataset(sfile)
#case2
path_2 = path + 'archive/case2/lnd/hist/'
#case3
path_3 = path + 'archive/case3/lnd/hist/'
#case4
path_4 = path + 'archive/case4/lnd/hist/'
#case5
path_5 = path + 'archive/case5/lnd/hist/'

# dynamic roof
sfile_dyn_rf = path + 'cesm_inputdata/lnd/clm2/urbandata/CLM50_DynUrbanAlbedoRoof_YuanSun_2023_0.9x1.25_simyr1849-2106_c20231005.nc'

start_year = 2015
end_year = 2100

In [ ]:
lsmmask = np.any(sf['PCT_URBAN'] != 0, axis=0)
mask = lsmmask.rename({'lsmlat': 'lat', 'lsmlon': 'lon'})

## dynamic albedo at grid-cell level

In [137]:
dyn_rf = xr.open_dataset(sfile_dyn_rf).sel(time = slice('2015-01-01 00:00:00', '2100-01-01 00:00:00'))
dyn_rf['time'] = dyn_rf['time'].dt.year

roof = (sf['WTLUNIT_ROOF']).where(lsmmask)
perroad = ((1 - sf['WTLUNIT_ROOF']) * sf['WTROAD_PERV']).where(lsmmask)
improad = ((1 - sf['WTLUNIT_ROOF']) * (1-sf['WTROAD_PERV'])).where(lsmmask)

alb_perroad = (sf['ALB_PERROAD_DIF'][0,:,:,:]).where(lsmmask)
alb_improad = (sf['ALB_IMPROAD_DIF'][0,:,:,:]).where(lsmmask)
alb_rf_tbd = (dyn_rf['dyn_alb_roof_TBD']).where(mask)
alb_rf_hd = (dyn_rf['dyn_alb_roof_HD']).where(mask)
alb_rf_md = (dyn_rf['dyn_alb_roof_MD']).where(mask)

In [138]:
arearoof = roof.mean()
areaperroad = perroad.mean()
areaimproad = improad.mean() 
print(arearoof, areaperroad, areaimproad)

<xarray.DataArray 'WTLUNIT_ROOF' ()>
array(0.54381721) <xarray.DataArray ()>
array(0.23361668) <xarray.DataArray ()>
array(0.22256611)


In [139]:
rf_tbd = (roof[0,:,:].rename({'lsmlat': 'lat', 'lsmlon': 'lon'})) * alb_rf_tbd 
rf_hd  = (roof[1,:,:].rename({'lsmlat': 'lat', 'lsmlon': 'lon'})) * alb_rf_hd 
rf_md  = (roof[2,:,:].rename({'lsmlat': 'lat', 'lsmlon': 'lon'})) * alb_rf_md
rf = xr.concat([rf_tbd, rf_hd, rf_md], dim='numurbl')
#rf # numurbl: 3lat: 192lon: 288time: 86

In [155]:
ipd = (perroad * alb_perroad).rename({'lsmlat': 'lat', 'lsmlon': 'lon'})
id = (improad * alb_improad).rename({'lsmlat': 'lat', 'lsmlon': 'lon'})
#alb = (rf + ipd + id)/(roof +perroad + improad).rename({'lsmlat': 'lat', 'lsmlon': 'lon'})
alb = (rf/roof.rename({'lsmlat': 'lat', 'lsmlon': 'lon'}))
alb # numurbl: 3lat: 192lon: 288time: 86

<xarray.DataArray (numurbl: 3, lat: 192, lon: 288, time: 86)>
array([[[[nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         ...,
         [nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan]],

        [[nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         ...,
         [nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan]],

        [[nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         ...,
...
         ...,
         [nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan]],

        [[nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         ...,
         [nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan]],

        [[nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         ...,
         [nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan]]]])
Coordinates:
  * lat      (lat) float64 -90.0 -89.06 -88.12 -87.17 ... 87.17 88.12 89.06 90.0
  * lon      (lon) float64 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.2 357.5 358.7
  * time     (time) int64 2015 2016 2017 2018 2019 ... 2096 2097 2098 2099 2100
Dimensions without coordinates: numurbl

In [156]:
df_alb = alb.mean(axis = 0).to_dataframe(name='alb').reset_index()
df_alb

,lat,lon,time,alb
0,-90.0,0.00,2015,NaN
1,-90.0,0.00,2016,NaN
2,-90.0,0.00,2017,NaN
3,-90.0,0.00,2018,NaN
4,-90.0,0.00,2019,NaN
...,...,...,...,...
4755451,90.0,358.75,2096,NaN
4755452,90.0,358.75,2097,NaN
4755453,90.0,358.75,2098,NaN
4755454,90.0,358.75,2099,NaN


In [157]:
df_alb.max()

lat       90.00
lon      358.75
time    2100.00
alb        0.90
dtype: float64

In [158]:
df_alb.rename(columns={'time': 'year'}, inplace=True)
df_alb['lsmlon'] = (df_alb['lon']/1.25).astype(int)
unique_lat_values = df_alb['lat'].unique()
lat_mapping = {lat: index for index, lat in enumerate(unique_lat_values)}
df_alb['lsmlat'] = df_alb['lat'].map(lat_mapping)
df_alb

,lat,lon,year,alb,lsmlon,lsmlat
0,-90.0,0.00,2015,NaN,0,0
1,-90.0,0.00,2016,NaN,0,0
2,-90.0,0.00,2017,NaN,0,0
3,-90.0,0.00,2018,NaN,0,0
4,-90.0,0.00,2019,NaN,0,0
...,...,...,...,...,...,...
4755451,90.0,358.75,2096,NaN,286,191
4755452,90.0,358.75,2097,NaN,286,191
4755453,90.0,358.75,2098,NaN,286,191
4755454,90.0,358.75,2099,NaN,286,191


In [159]:
sort = df_alb.dropna()
sort

,lat,lon,year,alb,lsmlon,lsmlat
936454,-55.130890,291.25,2015,0.394333,233,37
936455,-55.130890,291.25,2016,0.404333,233,37
936456,-55.130890,291.25,2017,0.414333,233,37
936457,-55.130890,291.25,2018,0.424333,233,37
936458,-55.130890,291.25,2019,0.434333,233,37
...,...,...,...,...,...,...
4212791,70.209424,31.25,2096,0.900000,25,170
4212792,70.209424,31.25,2097,0.900000,25,170
4212793,70.209424,31.25,2098,0.900000,25,170
4212794,70.209424,31.25,2099,0.900000,25,170


In [162]:
ALB = sort.groupby('year').mean().reset_index()['alb']
ALB.to_csv('gridcell_alb_dynroof.csv')

In [163]:
# check delta_albedo
df_sorted = sort.sort_values(by=['lat', 'lon', 'year'])
df_sorted['delta'] = df_sorted.groupby(['lat', 'lon'])['alb'].diff()
df_sorted

,lat,lon,year,alb,lsmlon,lsmlat,delta
936454,-55.130890,291.25,2015,0.394333,233,37,NaN
936455,-55.130890,291.25,2016,0.404333,233,37,0.01
936456,-55.130890,291.25,2017,0.414333,233,37,0.01
936457,-55.130890,291.25,2018,0.424333,233,37,0.01
936458,-55.130890,291.25,2019,0.434333,233,37,0.01
...,...,...,...,...,...,...,...
4212791,70.209424,31.25,2096,0.900000,25,170,0.00
4212792,70.209424,31.25,2097,0.900000,25,170,0.00
4212793,70.209424,31.25,2098,0.900000,25,170,0.00
4212794,70.209424,31.25,2099,0.900000,25,170,0.00


## Export heat flux variables

In [167]:
# case2 ann
case2_flux = pd.DataFrame()
for year in range(start_year,end_year):
    fn = 'case2.clm2.h1.' + '%04.0f' % year + '-02-01-00000.nc'
    if(os.path.exists(path_2+fn)):
        ds = xr.open_dataset(path_2+fn)
        df = (ds['FIRA_U']).mean('time').to_dataframe(name='FIRA_U').reset_index()
        df['year'] = year
        df['lsmlon'] = (df['lon']/1.25).astype(int)
        unique_lat_values = df['lat'].unique()
        lat_mapping = {lat: index for index, lat in enumerate(unique_lat_values)}
        df['lsmlat'] = df['lat'].map(lat_mapping)
        df = df.dropna()
        df['FSA_U'] = (ds['FSA_U']).mean('time').to_dataframe(name='FSA_U').reset_index()['FSA_U']
        df['FSH_U'] = (ds['FSH_U']).mean('time').to_dataframe(name='FSH_U').reset_index()['FSH_U']
        df['EFLX_LH_TOT_U'] = (ds['EFLX_LH_TOT_U']).mean('time').to_dataframe(name='EFLX_LH_TOT_U').reset_index()['EFLX_LH_TOT_U']
        df['FGR_U'] = (ds['FGR_U']).mean('time').to_dataframe(name='FGR_U').reset_index()['FGR_U']
        df['TG_U'] = (ds['TG_U']).mean('time').to_dataframe(name='TG_U').reset_index()['TG_U']
        df['FSDS'] = (ds['FSDS']).where(mask).mean('time').to_dataframe(name='FSDS').reset_index()['FSDS']
        df['FLDS'] = (ds['FLDS']).where(mask).mean('time').to_dataframe(name='FSDS').reset_index()['FSDS']
        case2_flux = pd.concat([case2_flux, df], ignore_index=True)

In [168]:
merged_df = pd.merge(case2_flux, sort, on=['lsmlat', 'lsmlon', 'year'], how='inner').dropna()
merged_df.to_csv('case2_export_annual.csv')

In [169]:
# case2 jja
case2_flux_jja = pd.DataFrame()
for year in range(start_year,end_year):
    fn = 'case2.clm2.h1.' + '%04.0f' % year + '-02-01-00000.nc'
    if(os.path.exists(path_2+fn)):
        ds = xr.open_dataset(path_2+fn)
        df = ds['FIRA_U'][6:9,:,:].mean('time').to_dataframe(name='FIRA_U').reset_index()
        df['year'] = year
        df['lsmlon'] = (df['lon']/1.25).astype(int)
        unique_lat_values = df['lat'].unique()
        lat_mapping = {lat: index for index, lat in enumerate(unique_lat_values)}
        df['lsmlat'] = df['lat'].map(lat_mapping)
        df = df.dropna()
        df['FSA_U'] = ds['FSA_U'][6:9,:,:].mean('time').to_dataframe(name='FSA_U').reset_index()['FSA_U']
        df['FSH_U'] = ds['FSH_U'][6:9,:,:].mean('time').to_dataframe(name='FSH_U').reset_index()['FSH_U']
        df['EFLX_LH_TOT_U'] = ds['EFLX_LH_TOT_U'][6:9,:,:].mean('time').to_dataframe(name='EFLX_LH_TOT_U').reset_index()['EFLX_LH_TOT_U']
        df['FGR_U'] = (ds['FGR_U'][6:9,:,:]).mean('time').to_dataframe(name='FGR_U').reset_index()['FGR_U']
        df['TG_U'] = (ds['TG_U'][6:9,:,:]).mean('time').to_dataframe(name='TG_U').reset_index()['TG_U']
        df['FSDS'] = (ds['FSDS'][6:9,:,:]).where(mask).mean('time').to_dataframe(name='FSDS').reset_index()['FSDS']
        df['FLDS'] = (ds['FLDS'][6:9,:,:]).where(mask).mean('time').to_dataframe(name='FSDS').reset_index()['FSDS']
        case2_flux_jja = pd.concat([case2_flux_jja, df], ignore_index=True)

In [170]:
merged_df_jja = pd.merge(case2_flux_jja, sort, on=['lsmlat', 'lsmlon', 'year'], how='inner').dropna()
merged_df_jja.to_csv('case2_export_jja.csv')

In [173]:
# case2 djf
case2_flux_djf = pd.DataFrame()
for year in range(start_year,end_year):
    fn = 'case2.clm2.h1.' + '%04.0f' % year + '-02-01-00000.nc'
    fn_1 = 'case2.clm2.h1.' + '%04.0f' % year + '-02-01-00000.nc'
    if(os.path.exists(path_2+fn)):
        ds = xr.open_dataset(path_2+fn)
        ds_1 = xr.open_dataset(path_2+fn)
        df = xr.concat([ds['FIRA_U'].isel(time=11),ds_1['FIRA_U'].isel(time=slice(0, 2))], dim='time').mean('time').to_dataframe(name='FIRA_U').reset_index()
        df['year'] = year
        df['lsmlon'] = (df['lon']/1.25).astype(int)
        unique_lat_values = df['lat'].unique()
        lat_mapping = {lat: index for index, lat in enumerate(unique_lat_values)}
        df['lsmlat'] = df['lat'].map(lat_mapping)
        df = df.dropna()
        df['FSA_U'] = xr.concat([ds['FSA_U'].isel(time=11),ds_1['FSA_U'].isel(time=slice(0, 2))], dim='time').mean('time').to_dataframe(name='FSA_U').reset_index()['FSA_U']
        df['FSH_U'] = xr.concat([ds['FSH_U'].isel(time=11),ds_1['FSH_U'].isel(time=slice(0, 2))], dim='time').mean('time').to_dataframe(name='FSH_U').reset_index()['FSH_U']
        df['EFLX_LH_TOT_U'] = xr.concat([ds['EFLX_LH_TOT_U'].isel(time=11),ds_1['EFLX_LH_TOT_U'].isel(time=slice(0, 2))], dim='time').mean('time').to_dataframe(name='EFLX_LH_TOT_U').reset_index()['EFLX_LH_TOT_U']
        df['FGR_U'] = xr.concat([ds['FGR_U'].isel(time=11),ds_1['FGR_U'].isel(time=slice(0, 2))], dim='time').mean('time').to_dataframe(name='FGR_U').reset_index()['FGR_U']
        df['TG_U'] = xr.concat([ds['TG_U'].isel(time=11),ds_1['FGR_U'].isel(time=slice(0, 2))], dim='time').mean('time').to_dataframe(name='TG_U').reset_index()['TG_U']
        df['FSDS'] = xr.concat([ds['FSDS'].isel(time=11),ds_1['FSDS'].isel(time=slice(0, 2))], dim='time').where(mask).mean('time').to_dataframe(name='FSDS').reset_index()['FSDS']
        df['FLDS'] = xr.concat([ds['FLDS'].isel(time=11),ds_1['FLDS'].isel(time=slice(0, 2))], dim='time').where(mask).mean('time').to_dataframe(name='FLDS').reset_index()['FLDS']
        case2_flux_djf = pd.concat([case2_flux_djf, df], ignore_index=True)

In [174]:
merged_df_djf = pd.merge(case2_flux_djf, sort, on=['lsmlat', 'lsmlon', 'year'], how='inner').dropna()
merged_df_djf.to_csv('case2_export_djf.csv')

In [180]:
# case3 ann
case3_flux = pd.DataFrame()
for year in range(start_year,end_year):
    fn = 'case3.clm2.h1.' + '%04.0f' % year + '-02-01-00000.nc'
    if(os.path.exists(path_3+fn)):
        ds = xr.open_dataset(path_3+fn)
        df = (ds['FIRA_U']).mean('time').to_dataframe(name='FIRA_U').reset_index()
        df['year'] = year
        df['lsmlon'] = (df['lon']/1.25).astype(int)
        unique_lat_values = df['lat'].unique()
        lat_mapping = {lat: index for index, lat in enumerate(unique_lat_values)}
        df['lsmlat'] = df['lat'].map(lat_mapping)
        df = df.dropna()
        df['FSA_U'] = (ds['FSA_U']).mean('time').to_dataframe(name='FSA_U').reset_index()['FSA_U']
        df['FSH_U'] = (ds['FSH_U']).mean('time').to_dataframe(name='FSH_U').reset_index()['FSH_U']
        df['EFLX_LH_TOT_U'] = (ds['EFLX_LH_TOT_U']).mean('time').to_dataframe(name='EFLX_LH_TOT_U').reset_index()['EFLX_LH_TOT_U']
        df['FGR_U'] = (ds['FGR_U']).mean('time').to_dataframe(name='FGR_U').reset_index()['FGR_U']
        df['TG_U'] = (ds['TG_U']).mean('time').to_dataframe(name='TG_U').reset_index()['TG_U']
        df['FSDS'] = (ds['FSDS']).where(mask).mean('time').to_dataframe(name='FSDS').reset_index()['FSDS']
        df['FLDS'] = (ds['FLDS']).where(mask).mean('time').to_dataframe(name='FSDS').reset_index()['FSDS']
        case3_flux = pd.concat([case3_flux, df], ignore_index=True)

In [181]:
merged_df = pd.merge(case3_flux, sort, on=['lsmlat', 'lsmlon', 'year'], how='inner').dropna()
merged_df.to_csv('case3_export_annual.csv')

In [177]:
# case4 ann
case4_flux = pd.DataFrame()
for year in range(start_year,end_year):
    fn = 'case4.clm2.h1.' + '%04.0f' % year + '-02-01-00000.nc'
    if(os.path.exists(path_4+fn)):
        ds = xr.open_dataset(path_4+fn)
        df = (ds['FIRA_U']).mean('time').to_dataframe(name='FIRA_U').reset_index()
        df['year'] = year
        df['lsmlon'] = (df['lon']/1.25).astype(int)
        unique_lat_values = df['lat'].unique()
        lat_mapping = {lat: index for index, lat in enumerate(unique_lat_values)}
        df['lsmlat'] = df['lat'].map(lat_mapping)
        df = df.dropna()
        df['FSA_U'] = (ds['FSA_U']).mean('time').to_dataframe(name='FSA_U').reset_index()['FSA_U']
        df['FSH_U'] = (ds['FSH_U']).mean('time').to_dataframe(name='FSH_U').reset_index()['FSH_U']
        df['EFLX_LH_TOT_U'] = (ds['EFLX_LH_TOT_U']).mean('time').to_dataframe(name='EFLX_LH_TOT_U').reset_index()['EFLX_LH_TOT_U']
        df['FGR_U'] = (ds['FGR_U']).mean('time').to_dataframe(name='FGR_U').reset_index()['FGR_U']
        df['TG_U'] = (ds['TG_U']).mean('time').to_dataframe(name='TG_U').reset_index()['TG_U']
        df['FSDS'] = (ds['FSDS']).where(mask).mean('time').to_dataframe(name='FSDS').reset_index()['FSDS']
        df['FLDS'] = (ds['FLDS']).where(mask).mean('time').to_dataframe(name='FSDS').reset_index()['FSDS']
        case4_flux = pd.concat([case4_flux, df], ignore_index=True)

In [178]:
merged_df = pd.merge(case4_flux, sort, on=['lsmlat', 'lsmlon', 'year'], how='inner').dropna()
merged_df.to_csv('case4_export_annual.csv')

In [183]:
# case5 ann
case5_flux = pd.DataFrame()
for year in range(start_year,end_year):
    fn = 'case5.clm2.h1.' + '%04.0f' % year + '-02-01-00000.nc'
    if(os.path.exists(path_5+fn)):
        ds = xr.open_dataset(path_5+fn)
        df = (ds['FIRA_U']).mean('time').to_dataframe(name='FIRA_U').reset_index()
        df['year'] = year
        df['lsmlon'] = (df['lon']/1.25).astype(int)
        unique_lat_values = df['lat'].unique()
        lat_mapping = {lat: index for index, lat in enumerate(unique_lat_values)}
        df['lsmlat'] = df['lat'].map(lat_mapping)
        df = df.dropna()
        df['FSA_U'] = (ds['FSA_U']).mean('time').to_dataframe(name='FSA_U').reset_index()['FSA_U']
        df['FSH_U'] = (ds['FSH_U']).mean('time').to_dataframe(name='FSH_U').reset_index()['FSH_U']
        df['EFLX_LH_TOT_U'] = (ds['EFLX_LH_TOT_U']).mean('time').to_dataframe(name='EFLX_LH_TOT_U').reset_index()['EFLX_LH_TOT_U']
        df['FGR_U'] = (ds['FGR_U']).mean('time').to_dataframe(name='FGR_U').reset_index()['FGR_U']
        df['TG_U'] = (ds['TG_U']).mean('time').to_dataframe(name='TG_U').reset_index()['TG_U']
        df['FSDS'] = (ds['FSDS']).where(mask).mean('time').to_dataframe(name='FSDS').reset_index()['FSDS']
        df['FLDS'] = (ds['FLDS']).where(mask).mean('time').to_dataframe(name='FSDS').reset_index()['FSDS']
        case5_flux = pd.concat([case5_flux, df], ignore_index=True)

In [184]:
merged_df = pd.merge(case5_flux, sort, on=['lsmlat', 'lsmlon', 'year'], how='inner').dropna()
merged_df.to_csv('case5_export_annual.csv')

# dynamic roof albedo at land-unit level

In [111]:
alb_rf_tbd['lndtype'] = 7
alb_rf_hd['lndtype'] = 8
alb_rf_md['lndtype'] = 9

df_alb_unit = pd.concat([alb_rf_tbd.to_dataframe(name='alb').reset_index(), 
                                  alb_rf_hd.to_dataframe(name='alb').reset_index(),
                                  alb_rf_md.to_dataframe(name='alb').reset_index()], ignore_index=True)

df_alb_unit.rename(columns={'time': 'year'}, inplace=True)
df_alb_unit['lsmlon'] = (df_alb_unit['lon']/1.25).astype(int)
unique_lat_values = df_alb_unit['lat'].unique()
lat_mapping = {lat: index for index, lat in enumerate(unique_lat_values)}
df_alb_unit['lsmlat'] = df_alb_unit['lat'].map(lat_mapping)

df_alb_3 = df_alb_unit.dropna()
df_alb_3

,year,lat,lon,lndtype,alb,lsmlon,lsmlat
10889,2015,-55.130890,291.25,7,0.263,233,37
11178,2015,-54.188482,292.50,7,0.263,234,38
11463,2015,-53.246073,288.75,7,0.263,230,39
11465,2015,-53.246073,291.25,7,0.263,233,39
11466,2015,-53.246073,292.50,7,0.263,234,39
...,...,...,...,...,...,...,...
14259770,2100,69.267016,32.50,9,0.900,26,169
14259771,2100,69.267016,33.75,9,0.900,27,169
14259814,2100,69.267016,87.50,9,0.900,69,169
14259815,2100,69.267016,88.75,9,0.900,71,169


In [112]:
# check delta_albedo
df_sorted3 = df_alb_3 .sort_values(by=['lat', 'lon', 'year', 'lndtype'])
df_sorted3['delta'] = df_sorted3.groupby(['lat', 'lon', 'lndtype'])['alb'].diff()
df_sorted3

,year,lat,lon,lndtype,alb,lsmlon,lsmlat,delta
10889,2015,-55.130890,291.25,7,0.263,233,37,NaN
4766345,2015,-55.130890,291.25,8,0.414,233,37,NaN
9521801,2015,-55.130890,291.25,9,0.506,233,37,NaN
66185,2016,-55.130890,291.25,7,0.273,233,37,0.01
4821641,2016,-55.130890,291.25,8,0.424,233,37,0.01
...,...,...,...,...,...,...,...,...
9449305,2099,70.209424,31.25,8,0.900,25,170,0.00
14204761,2099,70.209424,31.25,9,0.900,25,170,0.00
4749145,2100,70.209424,31.25,7,0.900,25,170,0.00
9504601,2100,70.209424,31.25,8,0.900,25,170,0.00


# Export land-unit level heat flux

In [121]:
# column level
# case2
case2_fira = pd.DataFrame()
for year in range(start_year, end_year):
    fn = 'case2.clm2.h0.' + '%04.0f' % year + '-02-01-00000.nc'
    if(os.path.exists(path_2+fn)):
        ds = xr.open_dataset(path_2+fn)
        lat = ds.lat
        lon = ds.lon
        time = ds.time
        ixy = ds.land1d_ixy
        jxy = ds.land1d_jxy
        landtype = ds.land1d_ityplunit
        nlat = len(lat.values)
        nlon = len(lon.values)
        ntim   = len(time.values)
        nland = int(np.max(landtype))+1
        U = ds['FIRA_U']
       
        grid_U = np.full([ntim,nland,nlat,nlon],np.nan)
        grid_U[:,landtype.values.astype(int),jxy.values.astype(int) - 1,ixy.values.astype(int) - 1] = U.values
        grid_U_dims = xr.DataArray(grid_U, dims=("time","lndtype","lat","lon"))
        grid_U_dims = grid_U_dims.assign_coords(time=ds.time,lndtype=[i for i in range(10)],lat=lat.values,lon=lon.values)
        
        df = grid_U_dims.sel(lndtype=slice(7,9)).mean('time').to_dataframe(name='FIRA_U').reset_index()
        df['year'] = year
        df['lsmlon'] = (df['lon']/1.25).astype(int)
        unique_lat_values_tbd = df['lat'].unique()
        lat_mapping_tbd = {lat: index for index, lat in enumerate(unique_lat_values_tbd)}
        df['lsmlat'] = df['lat'].map(lat_mapping_tbd)
        df = df.dropna()
        
        case2_fira = pd.concat([case2_fira, df], ignore_index=True)     

In [122]:
merged_fira = pd.merge(case2_fira, df_alb_3, on=['lsmlat', 'lsmlon', 'year', 'lndtype'], how='inner').dropna()
merged_fira.to_csv('case2_fira.csv')

In [123]:
case2_fsh = pd.DataFrame()
for year in range(start_year, end_year):
    fn = 'case2.clm2.h0.' + '%04.0f' % year + '-02-01-00000.nc'
    if(os.path.exists(path_2+fn)):
        ds = xr.open_dataset(path_2+fn)
        lat = ds.lat
        lon = ds.lon
        time = ds.time
        ixy = ds.land1d_ixy
        jxy = ds.land1d_jxy
        landtype = ds.land1d_ityplunit
        nlat = len(lat.values)
        nlon = len(lon.values)
        ntim   = len(time.values)
        nland = int(np.max(landtype))+1
        U = ds['FSH_U']
       
        grid_U = np.full([ntim,nland,nlat,nlon],np.nan)
        grid_U[:,landtype.values.astype(int),jxy.values.astype(int) - 1,ixy.values.astype(int) - 1] = U.values
        grid_U_dims = xr.DataArray(grid_U, dims=("time","lndtype","lat","lon"))
        grid_U_dims = grid_U_dims.assign_coords(time=ds.time,lndtype=[i for i in range(10)],lat=lat.values,lon=lon.values)
        
        df = grid_U_dims.sel(lndtype=slice(7,9)).mean('time').to_dataframe(name='FSH_U').reset_index()
        df['year'] = year
        df['lsmlon'] = (df['lon']/1.25).astype(int)
        unique_lat_values_tbd = df['lat'].unique()
        lat_mapping_tbd = {lat: index for index, lat in enumerate(unique_lat_values_tbd)}
        df['lsmlat'] = df['lat'].map(lat_mapping_tbd)
        df = df.dropna()
        
        case2_fsh = pd.concat([case2_fsh, df], ignore_index=True)

In [124]:
merged_fsh = pd.merge(case2_fsh, df_alb_3, on=['lsmlat', 'lsmlon', 'year', 'lndtype'], how='inner').dropna()
merged_fsh.to_csv('case2_fsh.csv')

In [125]:
case2_fsa = pd.DataFrame()
for year in range(start_year, end_year):
    fn = 'case2.clm2.h0.' + '%04.0f' % year + '-02-01-00000.nc'
    if(os.path.exists(path_2+fn)):
        ds = xr.open_dataset(path_2+fn)
        lat = ds.lat
        lon = ds.lon
        time = ds.time
        ixy = ds.land1d_ixy
        jxy = ds.land1d_jxy
        landtype = ds.land1d_ityplunit
        nlat = len(lat.values)
        nlon = len(lon.values)
        ntim   = len(time.values)
        nland = int(np.max(landtype))+1
        U = ds['FSA_U']
       
        grid_U = np.full([ntim,nland,nlat,nlon],np.nan)
        grid_U[:,landtype.values.astype(int),jxy.values.astype(int) - 1,ixy.values.astype(int) - 1] = U.values
        grid_U_dims = xr.DataArray(grid_U, dims=("time","lndtype","lat","lon"))
        grid_U_dims = grid_U_dims.assign_coords(time=ds.time,lndtype=[i for i in range(10)],lat=lat.values,lon=lon.values)
        
        df = grid_U_dims.sel(lndtype=slice(7,9)).mean('time').to_dataframe(name='FSA_U').reset_index()
        df['year'] = year
        df['lsmlon'] = (df['lon']/1.25).astype(int)
        unique_lat_values_tbd = df['lat'].unique()
        lat_mapping_tbd = {lat: index for index, lat in enumerate(unique_lat_values_tbd)}
        df['lsmlat'] = df['lat'].map(lat_mapping_tbd)
        df = df.dropna()
        
        case2_fsa = pd.concat([case2_fsa, df], ignore_index=True)

In [126]:
merged_fsa = pd.merge(case2_fsa, df_alb_3, on=['lsmlat', 'lsmlon', 'year', 'lndtype'], how='inner').dropna()
merged_fsa.to_csv('case2_fsa.csv')

In [127]:
case2_eflx = pd.DataFrame()
for year in range(start_year, end_year):
    fn = 'case2.clm2.h0.' + '%04.0f' % year + '-02-01-00000.nc'
    if(os.path.exists(path_2+fn)):
        ds = xr.open_dataset(path_2+fn)
        lat = ds.lat
        lon = ds.lon
        time = ds.time
        ixy = ds.land1d_ixy
        jxy = ds.land1d_jxy
        landtype = ds.land1d_ityplunit
        nlat = len(lat.values)
        nlon = len(lon.values)
        ntim   = len(time.values)
        nland = int(np.max(landtype))+1
        U = ds['EFLX_LH_TOT_U']
       
        grid_U = np.full([ntim,nland,nlat,nlon],np.nan)
        grid_U[:,landtype.values.astype(int),jxy.values.astype(int) - 1,ixy.values.astype(int) - 1] = U.values
        grid_U_dims = xr.DataArray(grid_U, dims=("time","lndtype","lat","lon"))
        grid_U_dims = grid_U_dims.assign_coords(time=ds.time,lndtype=[i for i in range(10)],lat=lat.values,lon=lon.values)
        
        df = grid_U_dims.sel(lndtype=slice(7,9)).mean('time').to_dataframe(name='EFLX_LH_TOT_U').reset_index()
        df['year'] = year
        df['lsmlon'] = (df['lon']/1.25).astype(int)
        unique_lat_values_tbd = df['lat'].unique()
        lat_mapping_tbd = {lat: index for index, lat in enumerate(unique_lat_values_tbd)}
        df['lsmlat'] = df['lat'].map(lat_mapping_tbd)
        df = df.dropna()
        
        case2_eflx = pd.concat([case2_eflx, df], ignore_index=True)

In [128]:
merged_eflx = pd.merge(case2_eflx, df_alb_3, on=['lsmlat', 'lsmlon', 'year', 'lndtype'], how='inner').dropna()
merged_eflx.to_csv('case2_eflx.csv')

In [129]:
case2_fgr = pd.DataFrame()
for year in range(start_year, end_year):
    fn = 'case2.clm2.h0.' + '%04.0f' % year + '-02-01-00000.nc'
    if(os.path.exists(path_2+fn)):
        ds = xr.open_dataset(path_2+fn)
        lat = ds.lat
        lon = ds.lon
        time = ds.time
        ixy = ds.land1d_ixy
        jxy = ds.land1d_jxy
        landtype = ds.land1d_ityplunit
        nlat = len(lat.values)
        nlon = len(lon.values)
        ntim   = len(time.values)
        nland = int(np.max(landtype))+1
        U = ds['FGR_U']
       
        grid_U = np.full([ntim,nland,nlat,nlon],np.nan)
        grid_U[:,landtype.values.astype(int),jxy.values.astype(int) - 1,ixy.values.astype(int) - 1] = U.values
        grid_U_dims = xr.DataArray(grid_U, dims=("time","lndtype","lat","lon"))
        grid_U_dims = grid_U_dims.assign_coords(time=ds.time,lndtype=[i for i in range(10)],lat=lat.values,lon=lon.values)
        
        df = grid_U_dims.sel(lndtype=slice(7,9)).mean('time').to_dataframe(name='FGR_U').reset_index()
        df['year'] = year
        df['lsmlon'] = (df['lon']/1.25).astype(int)
        unique_lat_values_tbd = df['lat'].unique()
        lat_mapping_tbd = {lat: index for index, lat in enumerate(unique_lat_values_tbd)}
        df['lsmlat'] = df['lat'].map(lat_mapping_tbd)
        df = df.dropna()
        
        case2_fgr = pd.concat([case2_fgr, df], ignore_index=True)

In [130]:
merged_fgr = pd.merge(case2_fgr, df_alb_3, on=['lsmlat', 'lsmlon', 'year', 'lndtype'], how='inner').dropna()
merged_fgr.to_csv('case2_fgr.csv')